<a href="https://colab.research.google.com/github/javst42/COVID-19-Datathon/blob/master/ProbDeath_GBM_SophiaH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# packages --------------------------------------------------------------
install.packages('rsample')
install.packages('caret')
install.packages('ggthemes')
install.packages('scales')
install.packages('wesanderson')
install.packages('gbm')
install.packages('Metrics')
install.packages('here')
install.packages('tidyverse')

library(rsample)      
library(caret)        
library(ggthemes)
#library(scales)
library(wesanderson)
library(tidyverse)
library(gbm)
library(Metrics)
library(here)

install.packages('pdp')
library(pdp)

In [0]:
df<- read.csv("survival_master.csv", header=T, na.strings = c("","NA","15-88", "18-50", "18-65"))
df_dummy <- read.csv('survival_master_dummy.csv', header = T, na.strings = c("","NA","15-88", "18-50", "18-65"))

df_dummy <- df_dummy %>% 
              mutate(date_onset_symptoms = ifelse(is.na(date_onset_symptoms) == T & is.na(date_confirmation)==F,
                                                  as.character(date_confirmation),
                                                  as.character(date_onset_symptoms)))

df_dummy <- df_dummy %>% 
  dplyr::mutate(date_onset_symptoms = as.Date(as.character(date_onset_symptoms),"%Y-%m-%d"),
                date_admission_hospital = as.Date(as.character(date_admission_hospital),"%Y-%m-%d"),
                date_death_or_discharge = as.Date(as.character(date_death_or_discharge),"%Y-%m-%d"),
                #age2 = case_when(
                #  age == '<10' ~ 5,
                #  age == '0-10' ~ 5,
                #  age == '0-6' ~ 3,
                #  age == '13-19' ~ 16,
                #  age == '20-29' ~ 25,
                #  age == '27-40' ~ 35,
                #  age == '30-39' ~ 35,
                #  age == '36-45' ~ 40,
                #  age == '40-49' ~ 45,
                #  age == '50-59' ~ 55,
                #  age == '60-69' ~ 65,
                #  age == '60-60' ~ 60,
                #  age == '70-79' ~ 75,
                #  age == '80-89' ~ 85,
                #  age == '90-99' ~ 95,
                #  age == '10s' ~ 15,
                #  age == '20s' ~ 25,
                #  age == '30s' ~ 35,
                #  age == '40s' ~ 45,
                #  age == '50s' ~ 55,
                #  age == '60s' ~ 65,
                #  age == '70s' ~ 75,
                #  age == '80s' ~ 85,
                #  age == '90s' ~ 95,
                #  age == 'elderly' ~ 95
                #),
                outcome2 = case_when(
                  #outcome == '05.02.2020' ~ NA,
                  outcome == 'dead' ~ 'died',
                  outcome == 'death' ~ 'died',
                  outcome == 'Deceased' ~ 'died',
                  outcome =='died' ~ 'died',
                  outcome == 'discharge' ~ 'discharged',
                  outcome == 'Discharged' ~ 'discharged',
                  outcome =='Recovered' ~ 'discharged',
                  outcome =='recovered' ~ 'discharged',
                  outcome == 'discharged' ~ 'discharged',
                  TRUE ~ ''),
                Admission = ifelse(is.na(date_admission_hospital) == FALSE, 1, 0),
                Recovery = ifelse(outcome2 == 'discharged', 1, 0),
                Death = ifelse(outcome2 == 'died', 1, 0),
                time_til_admission = as.numeric(difftime(date_admission_hospital, date_onset_symptoms, unit='days')),
                time_til_recovery = ifelse(outcome2 == 'discharged', difftime(date_death_or_discharge, date_onset_symptoms, unit='days'), NA),
                time_til_death = ifelse(outcome2 == 'died', difftime(date_death_or_discharge, date_onset_symptoms, unit='days'), NA)
  )

# time til hospital admission data
df_hosp = df_dummy %>% 
  dplyr::filter(is.na(date_onset_symptoms) == FALSE) %>%
  dplyr::filter(!(Admission == 0 & outcome2 %in% c('discharged'))) %>% 
  dplyr::filter(time_til_admission >= 0 | is.na(time_til_admission)) %>% 
  mutate(age_group = case_when(
    # age2 <= 10 ~ '(,10]',
    # age2 <= 20 ~ '(10,20]',
    age < 20 ~ '(,20)',
    age < 50 & age >=20 ~ '[20,50)',
    age <= 50 ~ '(,50]',
    #age2 <= 60 ~ '(50,60]',
    age <= 70 & age > 50 ~ '(50,70]',
    age >70  ~ '(70,+)'
    # age2 <= 90 ~ '(80,90]',
    # age2 >90 ~ '(90,+)'
  ))

# time til death/recovery data
df_rec = df_dummy %>% dplyr::filter(outcome2 == 'discharged', is.na(time_til_recovery) == FALSE)
df_died = df_dummy %>% dplyr::filter(outcome2 == 'died', is.na(time_til_death) == FALSE)

In [0]:
set.seed(123)
Split <- initial_split(df_dummy, prop = .7)
Train <- training(Split) %>% dplyr::select(Death,hospitalized,sex , age , cough , fever ,dyspnea,other_respiratory,fatigue,myalgia,other_neurological,emesis,other_systemic,other_gastrointestinal,asthenia,other_muscoloskeletal,pneumonia,rhinorrhea,pharyngitis,other_musculoskeletal,other_ocular,nausea,other_muscoluskeletal,malaise,ILI,pain,ARDS,sepsis,other_cardiovascular,AKD,`organ.failure`)
Test  <- testing(Split) %>% dplyr::select(Death,hospitalized,sex , age , cough , fever ,dyspnea,other_respiratory,fatigue,myalgia,other_neurological,emesis,other_systemic,other_gastrointestinal,asthenia,other_muscoloskeletal,pneumonia,rhinorrhea,pharyngitis,other_musculoskeletal,other_ocular,nausea,other_muscoluskeletal,malaise,ILI,pain,ARDS,sepsis,other_cardiovascular,AKD,`organ.failure`)

In [0]:
# model
set.seed(123)
death_gbm_fit <- gbm::gbm(Death ~ ., 
            distribution = "bernoulli",
             # the formula for the model (recall that the period means, "all 
             # variables in the data set")
             data = Train, 
             # data set
             verbose = TRUE, 
             # Logical indicating whether or not to print
             #  out progress and performance indicators
             shrinkage = 0.01, 
             # a shrinkage parameter applied to each tree in the expansion. 
             # Also known as the learning rate or step-size reduction; 0.001 
             # to 0.1 usually work, but a smaller learning rate typically 
             # requires more trees.
             interaction.depth = 3, 
             # Integer specifying the maximum depth of each tree (i.e., the 
             # highest level of variable interactions allowed). A value of 1 
             # implies an additive model, a value of 2 implies a model with up
             #  to 2-way interactions
             n.minobsinnode = 5,
             # Integer specifying the minimum number of observations in the 
             # terminal nodes of the trees. Note that this is the actual number 
             # of observations, not the total weight.
             n.trees = 500, 
             # Integer specifying the total number of trees to fit. This is 
             # equivalent to the number of iterations and the number of basis 
             # functions in the additive expansion.
             cv.folds = 5
             # Number of cross-validation folds to perform. If cv.folds>1 then
             # gbm, in addition to the usual fit, will perform a 
             # cross-validation, calculate an estimate of generalization error
             #  returned in cv.error
             )

In [0]:
# Tuning
# create hyperparameter grid
hyper_grid <- expand.grid(
  shrinkage = c(.01, .1, .3),
  interaction.depth = c(1, 3, 5),
  n.minobsinnode = c(5, 10, 15),
  bag.fraction = c(.65, .8, 1), 
  optimal_trees = 0,               # a place to dump results
  min_RMSE = 0                     # a place to dump results
)

# randomize data
random_index <- sample(1:nrow(Train), nrow(Train))
random_Train <- Train[random_index, ]

# grid search 
for(i in 1:nrow(hyper_grid)) {
  
  # reproducibility
  set.seed(123)
  
  # train model
  gbm.tune <- gbm::gbm(Death ~ ., 
            distribution = "bernoulli",
             # the formula for the model (recall that the period means, "all 
             # variables in the data set")
             data = random_Train, 
             # data set
            interaction.depth = hyper_grid$interaction.depth[i],
            shrinkage = hyper_grid$shrinkage[i],
            n.minobsinnode = hyper_grid$n.minobsinnode[i],
            bag.fraction = hyper_grid$bag.fraction[i],
            train.fraction = .75,
            n.cores = NULL, # will use all cores by default
            verbose = FALSE
             )
  
  # add min training error and trees to grid
  hyper_grid$optimal_trees[i] <- which.min(gbm.tune$valid.error)
  hyper_grid$min_RMSE[i] <- sqrt(min(gbm.tune$valid.error))
}

hyper_grid %>% 
  dplyr::arrange(min_RMSE) %>%
  head(10)

In [0]:
# update tuning parameters
hyper_grid <- expand.grid(
  shrinkage = c(.01, .1, .3),
  interaction.depth = c(1, 3, 5),
  n.minobsinnode = c(5, 10, 15),
  bag.fraction = c(.65, .8, 1), 
  optimal_trees = 0,               # a place to dump results
  min_RMSE = 0                     # a place to dump results
)

# grid search 
for(i in 1:nrow(hyper_grid)) {
  
  # reproducibility
  set.seed(123)
  
  # train model
  gbm.tune <- gbm::gbm(Death ~ ., 
            distribution = "bernoulli",
             # the formula for the model (recall that the period means, "all 
             # variables in the data set")
             data = random_Train, 
             # data set
             n.trees = 500,
            interaction.depth = hyper_grid$interaction.depth[i],
            shrinkage = hyper_grid$shrinkage[i],
            n.minobsinnode = hyper_grid$n.minobsinnode[i],
            bag.fraction = hyper_grid$bag.fraction[i],
            train.fraction = .75,
            n.cores = NULL, # will use all cores by default
            verbose = FALSE
             )
  
  # add min training error and trees to grid
  hyper_grid$optimal_trees[i] <- which.min(gbm.tune$valid.error)
  hyper_grid$min_RMSE[i] <- sqrt(min(gbm.tune$valid.error))
}

hyper_grid %>% 
  dplyr::arrange(min_RMSE) %>%
  head(10)

In [0]:
# for reproducibility
set.seed(123)

# train GBM model
gbm.fit.final <- gbm(
  formula = Death ~ .,
  distribution = "bernoulli",
  data = random_Train,
  n.trees = 57,
  interaction.depth = 5,
  shrinkage = 0.1,
  n.minobsinnode = 15,
  bag.fraction = .8, 
  train.fraction = 1,
  n.cores = NULL, # will use all cores by default
  verbose = FALSE
  )  

summary(
  gbm.fit.final, 
  cBars = 10,
  method = relative.influence, # also can use permutation.test.gbm
  las = 2
  )

In [0]:
# PDP 
# hospitalized
partial(gbm.fit.final, pred.var = c("hospitalized","age"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

partial(gbm.fit.final, pred.var = c("hospitalized","fever"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

partial(gbm.fit.final, pred.var = c("hospitalized","dyspnea"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

partial(gbm.fit.final, pred.var = c("hospitalized","cough"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

# age
partial(gbm.fit.final, pred.var = c("age","fever"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

partial(gbm.fit.final, pred.var = c("age","dyspnea"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

partial(gbm.fit.final, pred.var = c("age","cough"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

# symptoms
partial(gbm.fit.final, pred.var = c("fever","cough"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()

partial(gbm.fit.final, pred.var = c("dyspnea","cough"), n.trees = gbm.fit.final$n.trees, grid.resolution = 100, prob=T) %>%
  autoplot(rug = TRUE, train = random_Train) +
  scale_y_continuous()